In [1]:
#잡음 신호 csv 텍스트 파일 생성기
import random
length = 720 * 720 + 1000
f = open("noise.txt", 'w')

for i in range(length):
    f.write(str(random.random())+",")
f.write(str(random.random()))    
f.close()

In [7]:
%%timeit -n 1 -r 10
import csv
#잡음신호 가져오기
def csv2list(s):
    file = open(s, 'r')
    csvfile = csv.reader(file)
    lists = []
    for item in csvfile:
        lists.append(item)
    return lists[0]

#신호 발생기
length = 720 * 720 + 1000
caos_num = 3.98
ratio = 0.3
x1 = 0.0437853
x2 = 0.7324354


listr1 = [] #혼돈계 1
listr2 = [] #혼돈계 2
rn = csv2list('noise.txt')

for i in rn:
    k = float(i) #0~1사이의 어떤 신호
    x1 = caos_num * (ratio * k + x1 - x1 * ratio) * (1 - (ratio * k + x1 - x1 * ratio))
    x2 = caos_num * (ratio * k + x2 - x2 * ratio) * (1 - (ratio * k + x2 - x2 * ratio))
    listr1.append(x1)
    listr2.append(x2)

#x1-x2
result = []
for i in range(length):
    result.append(listr1[i]-listr2[i])

a = 0  
#동기화 시작값 탐색  
for i in range(length):
    if result[i] == 0:
        print("동기화 시작 부분: ", i)
        start = i
        a = 1
        break

#동기화가 되지 않을때
if a != 1:
    print("동기화 부분 없음")
#혼돈신호 가공(배치를 위한)
NoJungBok = [] #가공된 신호
for i in listr1[:72 * 72 + 500]: #500은 중복될 것으로 예상한 개수
    if i not in NoJungBok:
        NoJungBok.append(i)
NoJungBok = NoJungBok[:72*72] #개수에 맞게 슬라이싱

sortedNJB = sorted(NoJungBok)
for i in NoJungBok: #순서 찾기
    NoJungBok[NoJungBok.index(i)] = sortedNJB.index(i) #소수값 대신 순서값 넣어주기

#모듈 임포트================================
import cv2
import numpy as np

#이미지 불러오기
img = cv2.imread(r'./image/jennie.png')
img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

#cv2.imshow('h', img[0:10, 0:10])
#암호화될 이미지를 넣을 이미지 만들기
enc = np.zeros((720,720), dtype=np.uint8)

#각 영역에 숫자 부여하기
area = {}
cnt = 0
for i in range(0,720,10):
    for j in range(0, 720, 10):
        area[NoJungBok[cnt]]=img[j:j+10, i:i+10]
        cnt += 1    

#cv2.imshow('h', area[NoJungBok[0]])
print(NoJungBok[0])
#재배치
cnt =0
for i in range(0,720,10):
    for j in range(0, 720, 10):
        enc[j:j+10, i:i+10] = area[cnt]
        cnt += 1

"""
import numpy as np
#혼돈신호 가져오기
chaos = listr1[start+1:start+720*720+1]

#혼돈신호를 8비트로 변환하기
def chant(num):
    for i in range(256):
        if num >= i / 250 and num < (i+1) / 250:
            return i

#암호화 전의 이미지 출력
#cv2.imshow('before', enc)

#이미지 한줄로 만들기
listimg = []
for j in range(720):
    for i in range(720):
        listimg.append(enc[j,i])

#XOR로 암호화 하기
for i in range(720*720):
        listimg[i] = (listimg[i] ^ chant(chaos[i]))

#이미지리스트 배열로 만들기
for i in range(720):
    enc[i] = listimg[i*720:i*720+720]

#cv2.imshow('after', enc)
#cv2.waitKey(0)
#cv2.destroyAllWindows()
"""

동기화 시작 부분:  391
1022
동기화 시작 부분:  391
1022
동기화 시작 부분:  391
1022
동기화 시작 부분:  391
1022
동기화 시작 부분:  391
1022
동기화 시작 부분:  391
1022
동기화 시작 부분:  391
1022
동기화 시작 부분:  391
1022
동기화 시작 부분:  391
1022
동기화 시작 부분:  391
1022
1.58 s ± 39 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)
